In [1]:
from larq_models import LarqBiGCN
from larq_layers import LarqBiGCNConv
import tensorflow as tf
import larq as lq
from larq_testing import prepare_dataset, single_run, prepare_model, mask_to_weights
from spektral.data import SingleLoader
import datetime

In [2]:
dataset = prepare_dataset("Cora")
model = prepare_model(
            channels=64, 
            n_labels=7, 
            layers=2, 
            dropout=0.4, 
            weight_decay=0.00,
            input_quantizer="magnitude_aware_sign",
            kernel_quantizer="magnitude_aware_sign",
            model_name="LarqBiGCN",
            use_bias=False,
        )
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    patience=100, restore_best_weights=True)
loss_function = tf.keras.losses.CategoricalCrossentropy(
        from_logits=False, reduction="sum")
model.compile(optimizer=optimizer, loss=loss_function,
                  weighted_metrics=["accuracy"])
weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)
loader_te = SingleLoader(dataset, sample_weights=weights_te)

Pre-processing node features
Size of train set: 140
Size of val set: 500
Size of test set: 1000
Num classes: 7
Num features: 1433
Create a 2-layered Bi-GCN.
Layer 0, units=64
Layer 1, units=7


/Users/tlong/.pyenv/versions/3.9.1/envs/bi-gcn/lib/python3.9/site-packages/spektral/datasets/citation.py:108: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  a = nx.adjacency_matrix(nx.from_dict_of_lists(graph))  # CSR
/Users/tlong/.pyenv/versions/3.9.1/envs/bi-gcn/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [4]:
history = model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=1000,
    callbacks=[early_stopping_callback],
)

Epoch 1/1000
1/1 [==============================] - 0s 22ms/step - loss: 1.9459 - accuracy: 0.1429 - val_loss: 1.9458 - val_accuracy: 0.1140
Epoch 2/1000
1/1 [==============================] - 0s 14ms/step - loss: 1.9459 - accuracy: 0.1429 - val_loss: 1.9458 - val_accuracy: 0.1140
Epoch 3/1000
1/1 [==============================] - 0s 15ms/step - loss: 1.9459 - accuracy: 0.1429 - val_loss: 1.9458 - val_accuracy: 0.1140
Epoch 4/1000
1/1 [==============================] - 0s 14ms/step - loss: 1.9459 - accuracy: 0.1429 - val_loss: 1.9458 - val_accuracy: 0.1140
Epoch 5/1000
1/1 [==============================] - 0s 14ms/step - loss: 1.9459 - accuracy: 0.1429 - val_loss: 1.9458 - val_accuracy: 0.1140
Epoch 6/1000
1/1 [==============================] - 0s 13ms/step - loss: 1.9459 - accuracy: 0.1429 - val_loss: 1.9458 - val_accuracy: 0.1620
Epoch 7/1000
1/1 [==============================] - 0s 14ms/step - loss: 1.9459 - accuracy: 0.1429 - val_loss: 1.9459 - val_accuracy: 0.1620
Epoch 8/1000


In [5]:
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

import larq as lq

from spektral.datasets.citation import Cora
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import AdjToSpTensor, LayerPreprocess
from spektral.utils import tic, toc

class GraphConv(tf.keras.layers.Layer):
    def __init__(self, a):
        super(GraphConv, self).__init__()
        self.a = a

    def get_config(self):
        config = super().get_config()
        return config

    def call(self, inputs):
        return tf.matmul(a, inputs)


# Load data
dataset = Cora(normalize_x=True, transforms=[LayerPreprocess(GCNConv), AdjToSpTensor()])
graph = dataset[0]
x, a, y = graph.x, graph.a, graph.y
a = tf.sparse.to_dense(a)
mask_tr, mask_va, mask_te = dataset.mask_tr, dataset.mask_va, dataset.mask_te


Pre-processing node features


/Users/tlong/.pyenv/versions/3.9.1/envs/bi-gcn/lib/python3.9/site-packages/spektral/datasets/citation.py:108: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  a = nx.adjacency_matrix(nx.from_dict_of_lists(graph))  # CSR
/Users/tlong/.pyenv/versions/3.9.1/envs/bi-gcn/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [14]:
def generate_model(quantizer, channels, dropout_rate, layers=2, adj_input=False, **layer_kwargs):
    node_features = tf.keras.Input(shape=(x.shape[1],))
    x_intermediate = tf.keras.layers.BatchNormalization()(node_features)

    # Intermediate layers: Binary Activation - Dropout - Graph Convolution
    for layer in range(layers - 1):
        x_intermediate = lq.quantizers.MagnitudeAwareSign()(x_intermediate)
        x_intermediate = tf.keras.layers.Dropout(rate=dropout_rate)(x_intermediate)
        x_intermediate = lq.layers.QuantDense(
            units=channels, 
            kernel_initializer="he_uniform",
            kernel_quantizer=quantizer,
            **layer_kwargs)(x_intermediate)
        x_intermediate = GraphConv(a)(x_intermediate)

    # Final layer
    x_intermediate = lq.quantizers.MagnitudeAwareSign()(x_intermediate)
    x_intermediate = tf.keras.layers.Dropout(rate=dropout_rate)(x_intermediate)
    x_intermediate = lq.layers.QuantDense(
        units=dataset.n_labels, 
        kernel_initializer="he_uniform",
        kernel_quantizer=quantizer)(x_intermediate)
    x_intermediate = GraphConv(a)(x_intermediate)
    outputs = tf.keras.layers.Softmax()(x_intermediate)
    if adj_input:
        adj_matrix = tf.keras.layers.Input(shape=(a.shape[1],))
        model = tf.keras.Model(inputs=[node_features, adj_matrix], outputs=outputs, name="BiGCN")
    else:
        model = tf.keras.Model(inputs=node_features, outputs=outputs, name="BiGCN")
    intermediate_models = []
    for l in model.layers:
        if isinstance(l, lq.layers.QuantDense):
            intermediate_models.append(tf.keras.Model(inputs=node_features, outputs=l.output))
    return model, intermediate_models

In [9]:
model, intermediate_models = generate_model(quantizer="magnitude_aware_sign", dropout_rate=0., channels=64, layers=2)

optimizer = tf.keras.optimizers.experimental.AdamW(learning_rate=0.001, weight_decay=0.000)
#optimizer = lq.optimizers.Bop(gamma=0.001, threshold)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    patience=100, restore_best_weights=True)
early_stopping_callback.set_model(model)

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
accuracy_fn = tf.keras.metrics.CategoricalAccuracy()

class GradCallback(tf.keras.callbacks.TensorBoard):
    def __init__(self, log_dir="logs", histogram_freq=0, write_graph=True, write_images=False, write_steps_per_second=False, update_freq="epoch", profile_batch=0, embeddings_freq=0, embeddings_metadata=None, **kwargs):
        super().__init__(log_dir, histogram_freq, write_graph, write_images, write_steps_per_second, update_freq, profile_batch, embeddings_freq, embeddings_metadata, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch, logs)
        
        model = self.model

        model(x[None, :], training=False)

        with tf.GradientTape(persistent=True, watch_accessed_variables=True) as tape:
            tape.watch(model.trainable_variables)
            predictions = model(x[None, :], training=True)[0]
            loss = loss_fn(y[mask_tr], predictions[mask_tr])
            loss += sum(model.losses)
        grads = tape.gradient(loss, model.trainable_variables)
        tf.summary.experimental.set_step(epoch)

        intermediate_activations = []
        for intermediate_model in intermediate_models:
            intermediate_activations.append(intermediate_model(x[None, :]))

        with self._writers["train"].as_default():
            for g, var in zip(grads, model.trainable_variables):
                grad = g
                tf.summary.histogram(f"{var.name}_grad", grad)
            for i, activations in enumerate(intermediate_activations):
                tf.summary.histogram(f"activations_{i}", activations)
        self._writers["train"].flush()


callback = GradCallback(
    log_dir=f"tensorboard/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}",
    histogram_freq=5, write_images=True, update_freq="epoch", write_grads=True)
callback.set_model(model)

# Training step
@tf.function
def train():
    with tf.GradientTape() as tape:
        predictions = model(x[None, :], training=True)[0]
        loss = loss_fn(y[mask_tr], predictions[mask_tr])
        loss += sum(model.losses)
        accuracy = accuracy_fn(y[mask_tr], predictions[mask_tr])

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss

# val step
@tf.function
def validate():
    predictions = model(x[None, :], training=False)[0]
    loss = loss_fn(y[mask_va], predictions[mask_va])
    loss += sum(model.losses)
    accuracy = accuracy_fn(y[mask_va], predictions[mask_va])
    return loss, accuracy

# test step
@tf.function
def evaluate():
    predictions = model(x[None, :], training=False)[0]
    accuracy = accuracy_fn(y[mask_te], predictions[mask_te])
    return accuracy

callback.on_train_begin()
for epoch in range(1, 2000):
    callback.on_epoch_begin(epoch=epoch)
    train_loss = train()
    val_loss, val_acc = validate()
    print(f"Epoch {epoch} | train loss: {train_loss:.4f}, validation loss: {val_loss:.4f}, validation acc: {val_acc:.4f}", end="\r")
    callback.on_epoch_end(epoch=epoch)
callback.on_train_end()
print(f"Final loss = {train_loss}")
test_accuracy = evaluate()
print(f"Test accuracy = {test_accuracy:.4f}")

Using a binary weight quantizer without setting `kernel_constraint` may result in starved weights (where the gradient is always zero).
Using a binary weight quantizer without setting `kernel_constraint` may result in starved weights (where the gradient is always zero).


Final loss = 0.0008137446129694581alidation loss: 1.2541, validation acc: 0.7734
Test accuracy = 0.7734


In [20]:
model, intermediate_models = generate_model(quantizer="magnitude_aware_sign", dropout_rate=0.4, channels=64, layers=2, adj_input=True)

optimizer = tf.keras.optimizers.experimental.AdamW(learning_rate=0.001, weight_decay=0.0005)
#optimizer = lq.optimizers.Bop(gamma=0.001, threshold)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    patience=200, restore_best_weights=True)
early_stopping_callback.set_model(model)

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
accuracy_fn = tf.keras.metrics.CategoricalAccuracy()

model.compile(optimizer=optimizer, loss=loss_function, weighted_metrics=["accuracy"])
history = model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=1000,
    callbacks=[early_stopping_callback],
    verbose=1
)

Using a binary weight quantizer without setting `kernel_constraint` may result in starved weights (where the gradient is always zero).
Using a binary weight quantizer without setting `kernel_constraint` may result in starved weights (where the gradient is always zero).


Epoch 1/1000
1/1 [==============================] - 1s 638ms/step - loss: 1.9585 - accuracy: 0.1714 - val_loss: 1.9424 - val_accuracy: 0.3160
Epoch 2/1000
1/1 [==============================] - 0s 54ms/step - loss: 1.9480 - accuracy: 0.1643 - val_loss: 1.9408 - val_accuracy: 0.3160
Epoch 3/1000
1/1 [==============================] - 0s 56ms/step - loss: 1.9472 - accuracy: 0.1357 - val_loss: 1.9395 - val_accuracy: 0.2460
Epoch 4/1000
1/1 [==============================] - 0s 54ms/step - loss: 1.9471 - accuracy: 0.1714 - val_loss: 1.9384 - val_accuracy: 0.1220
Epoch 5/1000
1/1 [==============================] - 0s 54ms/step - loss: 1.9413 - accuracy: 0.1929 - val_loss: 1.9377 - val_accuracy: 0.1220
Epoch 6/1000
1/1 [==============================] - 0s 61ms/step - loss: 1.9427 - accuracy: 0.2000 - val_loss: 1.9372 - val_accuracy: 0.1220
Epoch 7/1000
1/1 [==============================] - 0s 51ms/step - loss: 1.9392 - accuracy: 0.1929 - val_loss: 1.9364 - val_accuracy: 0.1220
Epoch 8/1000

# Experiment 1: GCN baseline (no quantization)

# Experiment 2: Quantized GCN Baseline (Same as paper)

# Experiment 3: Dropout - reducing the dropout rate, removing dropout, and putting dropout before vs. after the binarization

# Experiment 4: Allow batchnorm to be trained

# Experiment 5: Batchnorm before every layer

# Experiment 6: Effect of l2 regularization, using binary regularizers

# Experiment 7: Hyperparameter tuning (allow the quantizers to change as part of this)

# Experiment 8: Increase number or width of layers (compare to standard GCN)

# Experiment 9: Use Bop Optimizer

# Experiment 10: Use two-part learning, start with full precision, then do second phase of binarized learning

# Experiment 11: Ternary version